### Experimental Parameters

In [1]:
import numpy as np
import pickle as pkl
%matplotlib inline

proposal_rates = np.array([1/300, 1/120, 1/60, 1/30, 1/10, 1/5, 1/2])
n_trials = 3

### Setup JSON File

In [2]:
import sys
import json

# Create experimental parameters and write to JSON file
max_block_size = 50
TX_rate = 5


params = {
    'Block tree proposal rate parameter': 0,
    'Block micro proposal rate parameter': 0, # 1 seconds
    'Block size (txs)': max_block_size,
    'Transaction rate parameter': TX_rate, # 1 second
    'Duration (sec)': 0,
    'Fork choice rule': 'BitcoinNG',
    'Network model': 'Decker-Wattenhorf',
    'Number of adversaries': 1,
    'Number of nodes': 100,
    'Probability of error in transaction confirmation': 0.01,
    'Transaction dataset': 'poisson',
    'Transaction scheduling rule': 'FIFO',
    'Logging enabled': True
 }

### Run experiments

In [ ]:
import os, shutil, pprint, glob, csv

os.chdir('/mnt/c/Users/Kelvin/Desktop/course_repo/Blockchain/blockchain-simulator/')

pp = pprint.PrettyPrinter()
os.system('touch results/bitcoin_ng.json')

import metrics
import uuid
from main import main

throughputs = {}
throughputs_unique = {}
main_chain_arrival_latencies = {}
finalization_latencies = {}

print(os.getcwd())
    
for i in range(0, len(proposal_rates)):
    rate = proposal_rates[i]
    params['Block tree proposal rate parameter'] = rate
    params['Block micro proposal rate parameter'] = 1/(10*rate)
    params['Duration (sec)'] = 50/rate
    d = {}
    d['setting-name'] = f'bitcoin-ng-test'
    d[f'bitcoin-ng-test'] = params
    print('Parameters:')
    pp.pprint(d)
    with open('results/bitcoin_ng.json', 'w+') as outfile:
        json.dump(d, outfile)
    throughputs[rate] = []
    throughputs_unique[rate] = []
    main_chain_arrival_latencies[rate] = []
    finalization_latencies[rate] = []
    for trial in range(0, n_trials):
        main('results/bitcoin_ng.json')
#         os.system('python3 main.py -f results/bitcoin_ng.json')
        throughput, throughput_unique = metrics.compute_throughputs(filename='results/bitcoin_ng.json')
        avg_main_chain_arrival_latency, avg_finalization_latency = metrics.compute_latency()
        
        
        shutil.copytree('./logs', './logs_{0}'.format(uuid.uuid4()) )
        
        throughputs[rate].append(throughput)
        throughputs_unique[rate].append(throughput_unique)
        main_chain_arrival_latencies[rate].append(avg_main_chain_arrival_latency)
        finalization_latencies[rate].append(avg_finalization_latency)

    
os.remove('results/bitcoin_ng.json')
print('Throughputs:\n', throughputs)
print('Unique Throughputs:\n', throughput_unique)
print('Main Chain Arrival Latencies:\n', main_chain_arrival_latencies)
print('Finalization Latencies:\n', finalization_latencies)

/mnt/c/Users/Kelvin/Desktop/course_repo/Blockchain/blockchain-simulator
Parameters:
{'bitcoin-ng-test': {'Block micro proposal rate parameter': 30.0,
                     'Block size (txs)': 50,
                     'Block tree proposal rate parameter': 0.0033333333333333335,
                     'Duration (sec)': 14999.999999999998,
                     'Fork choice rule': 'BitcoinNG',
                     'Logging enabled': True,
                     'Network model': 'Decker-Wattenhorf',
                     'Number of adversaries': 1,
                     'Number of nodes': 100,
                     'Probability of error in transaction confirmation': 0.01,
                     'Transaction dataset': 'poisson',
                     'Transaction rate parameter': 5,
                     'Transaction scheduling rule': 'FIFO'},
 'setting-name': 'bitcoin-ng-test'}


 54%|█████▎    | 40502/75604 [05:38<07:53, 74.15it/s] 

### Compute Averages

Average main chain latency is the time for a transaction to be added to the global blocktree - the timestamp the transaction was created.

Average finalization latency is the time for a transaction to be k blocks deep - to be added to the global blocktree.

In [ ]:
avg_main_chain_latencies = {}
for rate in main_chain_arrival_latencies:
    avg_main_chain_latencies[rate] = sum(main_chain_arrival_latencies[rate])/len(main_chain_arrival_latencies[rate])

print("Average Main Chain Arrival Latency:", avg_main_chain_latencies)
avg_finalization_latencies = {}
for rate in finalization_latencies:
    avg_finalization_latencies[rate] = sum(finalization_latencies[rate])/len(finalization_latencies[rate])
    
print("Average Finalization Latency:", avg_finalization_latencies)

avg_throughput = {}
for rate in throughputs:
    avg_throughput[rate] = sum(throughputs[rate])/len(throughputs[rate])
    
print("Average Throughput Latency:", avg_throughput)

avg_unique_throughput = {}
for rate in throughputs_unique:
    avg_unique_throughput[rate] = sum(throughputs_unique[rate])/len(throughputs_unique[rate])
    
print("Average Unique Throughput Latency:", avg_unique_throughput)

### Extract Finalization Depth and Delta for Blocks

In [ ]:
import csv
with open('logs/stats.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        if row[0]=='Finalization depth':
            k = int(row[1])
        elif row[0]=='Average network latency for blocks (sec)':
            delta = float(row[1])
print(delta)
print(k)

In [ ]:
bitcoin_ng_data = {}
bitcoin_ng_data['throughputs'] = throughputs
bitcoin_ng_data['throughputs_unique'] = throughputs_unique
bitcoin_ng_data['main_chain_arrival_latencies'] = main_chain_arrival_latencies
bitcoin_ng_data['finalization_latencies'] = finalization_latencies
bitcoin_ng_data['k'] = k
bitcoin_ng_data['delta'] = delta

pkl.dump(bitcoin_ng_data, open('bitcoin-ng-data.pkl', 'wb'))

In [ ]:
# bitcoin_ng_data = pkl.load( open( "bitcoin-ng-data-final.pkl", "rb" ) )
# throughputs = bitcoin_ng_data['throughputs']
# throughputs_unique = bitcoin_ng_data['throughputs_unique']
# main_chain_arrival_latencies = bitcoin_ng_data['main_chain_arrival_latencies']
# finalization_latencies = bitcoin_ng_data['finalization_latencies']
# k = bitcoin_ng_data['k']
# delta = bitcoin_ng_data['delta']
# max_block_size = 50

### Plotting the results

In [ ]:
import matplotlib.pylab as plt
x = avg_main_chain_latencies.keys()
y1 = avg_main_chain_latencies.values()
expected_y1 = (delta+1.0/proposal_rates)
y2 = avg_finalization_latencies.values()
expected_y2 = k*(delta+1.0/proposal_rates)

y3 = avg_unique_throughput.values()
T = list(map(lambda x: min(x, 10*max_block_size),  TX_rate/proposal_rates)) # transaction per block, 1 Tx per second / x blocks / sec = tx / block
expected_y3 = proposal_rates*T/(1+proposal_rates*delta)

plt.xlabel('Block Proposal Rate (blocks/sec)')
plt.ylabel('Main Chain Arrival Latency (sec)')
plt.title(f'Bitcoin-NG\nMain Chain Arrival Latency vs. Proposal Rate (k={k}, delta={delta})')
plt.grid(True)

plt.plot(x, y1, 'o-', label='observed rate')
plt.plot(x, expected_y1, 'o--', label='expected rate')
plt.legend()
plt.show()

In [ ]:
plt.xlabel('Block Proposal Rate (blocks/sec)')
plt.ylabel('Finalization Latency (sec)')
plt.title(f'Bitcoin-NG\nFinalization Latency vs. Proposal Rate (k={k}, delta={delta})')
plt.grid(True)

plt.plot(x, y2, 'o-', label='observed rate')
plt.plot(x, expected_y2, 'o--', label='expected rate')
plt.legend()
plt.show()

In [ ]:
plt.xlabel('Block Proposal Rate (blocks/sec)')
plt.ylabel('TX Throughput (Transaction/sec)')
plt.title(f'Bitcoin-NG\nThroughput vs. Proposal Rate (k={k}, delta={delta})')
plt.grid(True)


plt.plot(x, y3, 'o-', label='observed rate')
plt.plot(x, expected_y3, 'o--', label='expected rate')
plt.legend()
plt.ylim([0, TX_rate])
plt.show()
